# XG boost modelling by month for Time Series Store forecast 
* By Alex Dance https://www.linkedin.com/in/alex-dance/
* This notebook is one of several notebooks for a project to improve store and product forecasts
1.	EDA – Exploratory Data Analysis – includes working with annual forecasts
2.	Main Modelling
3.	XG Boost modelling by Month
4.	Weighted average
5.	ARIMA – Month and Other Modelling
6.	Deep Learning

* This workbook is XG Boost for further forecasts 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier


In [ ]:
df = pd.read_csv("../input/demand-forecasting-kernels-only/train.csv")
df.head()

In [ ]:
split = "2017-01-01"

In [ ]:
df['date'] =  pd.to_datetime(df['date'])

In [ ]:
df = df.set_index('date')

In [ ]:
df.head()

In [ ]:
df_XG = df.groupby('store').resample('M')['sales'].sum()

In [ ]:
df_XG_store_item = df.groupby(['store','item'])['sales'].resample('M').sum()

In [ ]:
df_XG.head()

In [ ]:
df_XG_store_item.tail()

In [ ]:
df_XG.head()

In [ ]:
df_XG_store_item.head()

In [ ]:
df_XG_store_item.tail()

In [ ]:
df_XG = df_XG.reset_index()

In [ ]:
df_XG.head()

In [ ]:
df_XG_store_item= df_XG_store_item.reset_index() 

In [ ]:
df_XG_store_item.head()

In [ ]:
df_XG = df_XG.set_index('date')

In [ ]:
df_XG_store_item = df_XG_store_item.set_index('date')

In [ ]:
df_XG_store_item.tail()

In [ ]:
df_XG.info()

In [ ]:
df_XG['month'] =df_XG.index.month
df_XG['year'] = df_XG.index.year

In [ ]:
df_XG_store_item['month'] =df_XG_store_item.index.month
df_XG_store_item['year'] = df_XG_store_item.index.year

In [ ]:
df_XG.head()

In [ ]:
df_train = df_XG[ :split ] 
df_test = df_XG[split : ] 

In [ ]:
df_train_SI = df_XG_store_item[ :split ] 
df_test_SI = df_XG_store_item[split : ] 

In [ ]:
y_train = df_train.loc[:,'sales']
y_test= df_test.loc[:,'sales']
X_train = df_train.drop (['sales'],axis=1)
X_test = df_test.drop (['sales'],axis=1)

In [ ]:
y_train_SI = df_train_SI.loc[:,'sales']
y_test_SI= df_test_SI.loc[:,'sales']
X_train_SI = df_train_SI.drop (['sales'],axis=1) 
X_test_SI = df_test_SI.drop (['sales'],axis=1)

In [ ]:
y_train_SI.tail()

In [ ]:
X_test_SI.tail()

# XG Boost by Month

In [ ]:
df_XG_month= df.resample('M')['sales'].sum()

In [ ]:
df_XG_month = df_XG_month.reset_index()

In [ ]:
df_XG_month['year'] = df_XG_month['date'].dt.year

In [ ]:
df_XG_month = df_XG_month.set_index('date')

In [ ]:
df_train_month = df_XG_month[ :split ] 
df_test_month = df_XG_month[split : ] 

In [ ]:
df_train_month.head()

In [ ]:
y_train_month = df_train_month.loc[:,'sales']
y_test_month= df_test_month.loc[:,'sales']
X_train_month = df_train_month.drop (['sales'],axis=1) 
X_test_month = df_test_month.drop (['sales'],axis=1)

In [ ]:
XG_model_month = xgb.XGBRegressor(n_estimators=1000)

In [ ]:
XG_model_month.fit(X_train_month, y_train_month,eval_set=[(X_test_month, y_test_month)], early_stopping_rounds=50,verbose=False) # Change verbose to True to see it train

In [ ]:
_ = plot_importance(XG_model_month, height=0.9)

In [ ]:
XG_test_prediction = XG_model_month.predict(X_test_month)

In [ ]:
print(XG_test_prediction)

In [ ]:
# You can see by looking at the results that this is a terrible forecast as there are not enough features

# XG Boost on Store by Month

In [ ]:
XG_model = xgb.XGBRegressor(n_estimators=1000)

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
%%time
XG_model.fit(X_train, y_train,eval_set=[(X_test, y_test)], early_stopping_rounds=50,verbose=False) # Change verbose to True to see it train

In [ ]:
_ = plot_importance(XG_model, height=0.9)

In [ ]:
XG_test_prediction = XG_model.predict(X_test)

In [ ]:
XG_test_all =X_test.copy()
XG_train_all =X_train.copy()
XG_test_all['XG prediction'] = XG_model.predict(X_test)
XG_train_all['XG prediction'] =XG_model.predict(X_train)
XG_test_all['sales'] = y_test
XG_train_all['sales'] = y_train
df_xg_all = pd.concat([XG_test_all, XG_train_all], sort=False)

In [ ]:
XG_test_all.sum()

In [ ]:
X_train.head()

In [ ]:
df_xg_all.head()

In [ ]:
y_train.head()

In [ ]:
df_xg_all.sample(10)

In [ ]:
XG_test_all.describe()

In [ ]:
XG_train_all.describe()

In [ ]:
XG_test_all['sales'].sum()

In [ ]:
XG_test_all['XG prediction'].sum()

In [ ]:
XG_RMSE  = np.mean(np.sqrt((XG_test_all['XG prediction'] - XG_test_all['sales']) ** 2))  

In [ ]:
print(XG_RMSE)

In [ ]:
XG_test_all.shape

In [ ]:
XG_test_all.head()

In [ ]:
df_xg_all.describe()

In [ ]:
_ = df_xg_all[['sales','XG prediction']].plot(figsize=(15, 5))
# as too many options the plot is not very useful

In [ ]:
df_xg_all_1 = df_xg_all[(df_xg_all.store==1)]

In [ ]:
df_xg_all_3 = df_xg_all[(df_xg_all.store==3)]

In [ ]:
XG_train_all_1 = XG_train_all[(XG_train_all.store==1)]

In [ ]:
_ = df_xg_all_1[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
_ = df_xg_all_3[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
df_xg_all.sample(10)

In [ ]:
df_xg_all.head()

# XG by Store and ITEM By Month

In [ ]:
X_test_SI.head()

In [ ]:
XG_model_SI = xgb.XGBRegressor(n_estimators=1000)

In [ ]:
%%time
XG_model_SI.fit(X_train_SI, y_train_SI,eval_set=[(X_test_SI, y_test_SI)], early_stopping_rounds=50,verbose=False) # Change verbose to True to see it train
#XG_model.fit(X_train, y_train,eval_set=[(X_test, y_test)]

In [ ]:
_ = plot_importance(XG_model_SI, height=0.9)

In [ ]:
XG_test_prediction_SI = XG_model_SI.predict(X_test_SI)

In [ ]:
XG_test_prediction_SI.sum()

In [ ]:
XG_SI_test_all =X_test_SI.copy()

In [ ]:
XG_SI_train_all =X_train_SI.copy()

In [ ]:
XG_SI_test_all['XG prediction'] = XG_model_SI.predict(X_test_SI)

In [ ]:
XG_SI_test_all['sales'] = y_test_SI

In [ ]:
XG_SI_train_all['sales'] = y_train_SI

In [ ]:
XG_SI__RMSE  = np.mean(np.sqrt((XG_SI_test_all['XG prediction'] - XG_SI_test_all['sales']) ** 2)) 

In [ ]:
print(XG_SI__RMSE) # This result is very good

In [ ]:
XG_SI_test_all.shape

In [ ]:
df_xg_SI_all = pd.concat([XG_SI_test_all, XG_SI_train_all], sort=False)

In [ ]:
XG_SI_test_all.sample(10)

In [ ]:
XG_SI_train_all.sample(3)

In [ ]:
df_xg_SI_all.sample(5)

In [ ]:
_ = df_xg_SI_all[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
df_xg_SI_all_1_1 = df_xg_SI_all[(df_xg_SI_all.store==1)&(df_xg_SI_all.item==1)]

In [ ]:
_ = df_xg_SI_all_1_1[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
df_xg_SI_all_1_2 = df_xg_SI_all[(df_xg_SI_all.store==1)&(df_xg_SI_all.item==2)]

In [ ]:
_ = df_xg_SI_all_1_2[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
df_xg_SI_all_2_2 = df_xg_SI_all[(df_xg_SI_all.store==2)&(df_xg_SI_all.item==2)]

In [ ]:
_ = df_xg_SI_all_2_2[['sales','XG prediction']].plot(figsize=(15, 5))

In [ ]:
df_xg_SI_all.nunique()

In [ ]:
df_xg_SI_all.store.value_counts()